## <CENTER>  Mätning av $Z$-bosonens massa med ATLAS-data

### Börja med att hämta datan och ladda in de i ROOT-träd.

In [1]:
import ROOT

Welcome to JupyROOT 6.14/04


In [35]:
# Leta på följande webbadress, där ni bör hitta två filer, en för händelser med elektroner och en för myoner
# http://opendata.atlas.cern/release/samples
data_file_electrons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataEgamma.root")
data_file_electrons_simulation = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147770.Zee.root")
data_file_muons = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/Data/DataMuons.root")
data_file_muons_simulation = ROOT.TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147771.Zmumu.root")

In [36]:
# Ladda in i träd (TFile::ls() kan användas för att lista filens innehåll)
tree_electrons = data_file_electrons.Get("mini")
tree_electrons_simulation = data_file_electrons_simulation.Get("mini")
tree_muons = data_file_muons.Get("mini")
tree_muons_simulation = data_file_muons_simulation.Get("mini")

In [28]:
tree_electrons_simulation.Print()

******************************************************************************
*Tree    :mini      : 4-vectors + variables required for scaling factors     *
*Entries :  7500000 : Total =      2134734254 bytes  File  Size =  966057844 *
*        :          : Tree compression factor =   2.21                       *
******************************************************************************
*Br    0 :runNumber : runNumber/I                                            *
*Entries :  7500000 : Total  Size=   30006373 bytes  File Size  =     152653 *
*Baskets :       63 : Basket Size=     808960 bytes  Compression= 196.56     *
*............................................................................*
*Br    1 :eventNumber : eventNumber/I                                        *
*Entries :  7500000 : Total  Size=   30006507 bytes  File Size  =   16864535 *
*Baskets :       63 : Basket Size=     808960 bytes  Compression=   1.78     *
*...................................................

### Funktioner för invariant massa

In [4]:
def getInvMass(lep1_pt, lep1_eta, lep1_phi, lep2_pt, lep2_eta, lep2_phi):
    ''' räkna ut invarianta massan med formeln i Introduktions-notboken (antar masslösa partiklar) '''
    import math
    msq = 2*lep1_pt*lep2_pt*(math.cosh(lep1_eta-lep2_eta) - math.cos(lep1_phi-lep2_phi))
    return math.sqrt(msq)

In [5]:
def getInvMassTL(lep1_pt, lep1_eta, lep1_phi, lep1_e, lep2_pt, lep2_eta, lep2_phi, lep2_e):
    ''' räkna ut invarianta massan med TLorentzVectors '''
    lep1=ROOT.TLorentzVector()
    lep2=ROOT.TLorentzVector()
    lep1.SetPtEtaPhiE(lep1_pt, lep1_eta, lep1_phi, lep1_e)
    lep2.SetPtEtaPhiE(lep2_pt, lep2_eta, lep2_phi, lep2_e)
    invmass=lep1+lep2
    return invmass.M()

### Funktion för att analysera event

In [39]:
# funktion för att analysera event
def analyseEvt(evt, hmass, particle_type):
    if evt.lep_n != 2: return
    if not (evt.lep_type[0]==particle_type and evt.lep_type[1]==particle_type): return 
    if (evt.lep_charge[0] + evt.lep_charge[1]) != 0: return
    # passed all cuts, fill histogram
    hmass.Fill(getInvMass(evt.lep_pt[0]*1e-3, evt.lep_eta[0], evt.lep_phi[0], evt.lep_pt[1]*1e-3, evt.lep_eta[1], evt.lep_phi[1]))

### Förbered histogram

In [40]:
# Definiera binnar
nbins=50
lowedge=30.
upedge=150.
# histogram för elektron-händelser och myon-händelser
h_mass_electrons = ROOT.TH1F("h_mass_electrons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons = ROOT.TH1F("h_mass_muons", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_electrons_simulation = ROOT.TH1F("h_mass_electrons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons_simulation = ROOT.TH1F("h_mass_muons_simulation", "; Invariant mass [GeV]; Number of events", nbins, lowedge, upedge)
h_mass_muons_simulation

### Kör event-loopen, fyll histogram

In [44]:
# Loopa över alla händelser, räkna ut den invarianta massan av elektron-positron-par och myon-antimyon-par
# Detta kan t.ex. göras med hjälp av ROOT-klassen TLorentzVector eller genom att skriva en egen funktion.
# Fyll histogram med den invarianta massan.
# Använd vad ni lärde dig i första Introduktion-notebook.
# Nu tittar ni på data, så varje händelse kommer inte att innehålla en Z-boson.
# Ni måste själva sortera ut de händelser som har de karakteristiska drag man förväntar sig för Z-bosoner.

nevents=4000
for ievt in range(nevents):
    tree_electrons.GetEntry(ievt)
    tree_muons.GetEntry(ievt)

    analyseEvt(tree_electrons, h_mass_electrons, 11) # 11 and 13 are electrons and muons, respectively
    analyseEvt(tree_muons, h_mass_muons,     13) # 11 and 13 are electrons and muons, respectively    
    ievt += 1
    

In [45]:
# Gör samma sak för MC, så kan vi rita upp den med
nevents=4000
for ievt in range(nevents):
    tree_electrons_simulation.GetEntry(ievt)
    tree_muons_simulation.GetEntry(ievt)

    analyseEvt(tree_electrons_simulation, h_mass_electrons_simulation, 11) # 11 and 13 are electrons and muons, respectively
    analyseEvt(tree_muons_simulation, h_mass_muons_simulation,     13) # 11 and 13 are electrons and muons, respectively    
    ievt += 1

### Jämför invariant-mass-peak för elektroner och myoner

In [46]:
h_mass_electrons_simulation.Draw("e1p")
h_mass_muons_simulation.Draw("e1p SAME")
canvas.Draw()



In [33]:
# Draw MET
met = ROOT.TH1F("MET_simu", "; Missing transverse energy [GeV]; Number of events", 30, 0., 90.)
tree_electrons_simulation.Draw("met_et*1e-3 >> MET_simu", "", "e1p", 5000)
met.Draw("e1p")

Warning in <TDavixFile::Append>: Replacing existing TH1: MET_simu (Potential memory leak).


In [34]:
canvas.Draw()

### Rita upp den invarianta massan

In [9]:
# %jsroot on

In [26]:
canvas = ROOT.TCanvas("canvas", "", 800, 600)
# Draw data
h_mass_electrons_simulation.Draw("e1p")

Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas


In [ ]:
# Draw MC
h_mass_electrons_simulation.Scale(0.5)
h_mass_electrons_simulation.SetFillColor(ROOT.kRed)
h_mass_electrons_simulation.SetMarkerColor(ROOT.kRed)
h_mass_electrons_simulation.SetLineColor(ROOT.kRed)
h_mass_electrons_simulation.Draw("he1 SAME")

In [27]:
canvas.Draw()

### Anpassning med normalkurva

In [ ]:
# Gör en anpassning till peaken med rätt funktion
# prova först dummy Gauss
h_mass_electrons.Fit("gaus", "S", "SAME", 80., 100.)

In [ ]:
canvas.Draw()

### Normalkurvan ser inte så bra ut, vi provar mer korrekt modell

In [13]:
#### Prepare: setting Breit-Wigner limits, colors on functions etc.
norm_low = 100; norm_up = 15e3
mean_low = 80; mean_up = 100
width_low = 0.1; width_up = 40
def setBWLims(func):
    ''' Set limits on BW parameters '''
    func.SetParameters(0.5*(norm_low+norm_up), 0.5*(mean_low+mean_up), 0.5*(width_low+width_up))
    func.SetParLimits(0, norm_low, norm_up)
    func.SetParLimits(1, mean_low, mean_up)
    func.SetParLimits(2, width_low, width_up)
def setColor(ROOT_obj, color=ROOT.kBlack):
    ''' set (line) color'''
    ROOT_obj.SetLineColor(color)

In [18]:
#### DEFINE SOME BREIT-WIGNER FUNCTIONS

xlow_fit=30.; xup_fit=140.;
#foo=ROOT.TMath.BreitWigner(4., 3., 3.)
fBW = ROOT.TF1("fBW", "[0]*TMath::BreitWigner(x, [1], [2])", xlow_fit, xup_fit)
setBWLims(fBW)
# declare its own function for signal
fSig = ROOT.TF1("fSig", "[0]*TMath::BreitWigner(x, [1], [2])", xlow_fit, xup_fit)
# first order polynomial as background
fFit1 = ROOT.TF1("fit1", "fBW+pol1(3)", xlow_fit, xup_fit)
setBWLims(fFit1)
# second order polynomial as background
fFit2 = ROOT.TF1("fit2", "fBW+pol2(3)", xlow_fit, xup_fit)
# second order polynomial only
fBkg2 = ROOT.TF1("bkg2", "pol2", xlow_fit, xup_fit)
setBWLims(fFit2)
# third order polynomial as background
fFit3 = ROOT.TF1("fit3", "fBW+pol3(3)", xlow_fit, xup_fit)
setBWLims(fFit3)
# constant plus exponential for bkg
fBkg = ROOT.TF1("bkg", "pol0+expo(1)", xlow_fit, xup_fit)
#setBkgLimits(fBkg)
fBkg.Print()
fModel = ROOT.TF1("model", "fBW+bkg", xlow_fit, xup_fit)
setBWLims(fModel)
# set color
setColor(fBkg, ROOT.kRed)
setColor(fFit1); setColor(fFit2); setColor(fFit3);
setColor(fBW, ROOT.kBlue)
#setColor(fBkg2, ROOT.kBlue)

Formula based function:     bkg 
                  bkg : pol0+expo(1) Ndim= 1, Npar= 3, Number= 0 
 Formula expression: 
	[p0]+exp([p1]+[p2]*x) 


### Anpassning med Breit-Wigner

In [23]:
%jsroot on

In [24]:
#canvas_fit = ROOT.TCanvas("canvas_fit", "", 800, 600) # open a new canvas
# canvas_fit.cd()
#canvas.Clear()
# h_mass_electrons.Draw("e1p")

#fitresult=h_mass_electrons.Fit("fit2", "S+", "SAME", xlow_fit, xup_fit) # store fit result
fitresult=h_mass_electrons_simulation.Fit("model", "S", "e1p", xlow_fit, xup_fit) # store fit result
# Draw signal and bkg on top, using fit result
fitresult_parameters={}
for i_p,p in enumerate(fitresult.Parameters()):
    fitresult_parameters[i_p]=(p, fitresult.ParError(i_p))
    
### signal
# for ipar in range(3): fSig.SetParameter(ipar, fitresult_parameters[ipar][0])
# fSig.Draw("SAME")
for ipar in range(3): fBW.SetParameter(ipar, fitresult_parameters[ipar][0])
fBW.Draw("SAME")

### bkg
for ipar in range(2): fBkg2.SetParameter(ipar, fitresult_parameters[ipar+3][0])
fBkg.Draw("SAME")
#fBkg2.Draw("SAME")
#ROOT.disableJSVis()
#ROOT.enableJSVis()
canvas.Draw()
#ROOT.disableJSVis()
#canvas.SaveAs("figures/fit.pdf")
#for obj in canvas.GetListOfPrimitives():
#    print(obj)

 FCN=397.398 FROM MIGRAD    STATUS=CONVERGED      80 CALLS          81 TOTAL
                     EDM=2.55918e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.39906e+04   1.99267e+02   4.76859e-04   6.52480e-04
   2  p1           8.98190e+01   6.03693e-02   5.20732e-05   6.23650e-03
   3  p2           5.37544e+00   1.17595e-01   6.74770e-05  -1.66100e-03
   4  p3          -1.95260e+01   2.76800e+01   3.86776e-03   1.46232e-04
   5  p4           3.18982e+00   1.09273e+00   2.28524e-04   2.46254e-06
   6  p5          -3.84189e-03   6.09849e-03   2.33992e-06   5.58098e-01


In [ ]:
# Draw signal and bkg on top, using fit result
fitresult_parameters={}
for i_p,p in enumerate(fitresult.Parameters()):
    fitresult_parameters[i_p]=(p, fitresult.ParError(i_p))
# signal
fBW.SetParameter(0, fitresult_parameters[0][0])
fBW.SetParameter(1, fitresult_parameters[1][0])
fBW.SetParameter(2, fitresult_parameters[2][0])
fBW.Draw("SAME")
fBW.Print()
print([fBW.GetParameter(i) for i in range(3)])
# bkg
fBkg2.SetParameter(0, fitresult_parameters[0+3][0])
fBkg2.SetParameter(1, fitresult_parameters[1+3][1])
fBkg2.Draw("SAME")

In [ ]:
canvas.Draw()

In [ ]:
### doesn't seem to work
#def getNumpyArrFromBuffer(buffer,size_buffer):
#    ''' get numpy array from buffer '''
#    import numpy
#    buffer.SetSize(size_buffer)
#    return numpy.array(buffer, copy=True)